In [1]:
# Imports
import random
import heapq

In [2]:
class Maze:
  """Класс Maze предоставляет удобный способ работы с лабиринтом.
  Он поддерживает методы:
  * загрузки из файла и сохранения в файл лабиринта;
  * проверки попадания точки в область лабиринта;
  * проверки наличия или отсутствия стены;
  * set и get методы для точки в лабиринте;
  * размеры лабиринта.
  """
  def __init__(self, file_name: str=None):
    self.maze = []
    if file_name:
      self.load_maze(file_name)

  def load_maze(self, file_name: str) -> None:
    with open(file_name, 'r') as file:
      self.maze = [list(line) for line in file.readlines()]

  def save_maze(self, file_name: str) -> None:
    with open(file_name, 'w') as file:
      file.writelines(''.join(line) for line in self.maze)

  def check_point(self, point: tuple) -> bool:
    return 0 <= point[0] < self.width() and 0 <= point[1] < self.height()

  def is_wall(self, point: tuple) -> bool:
    return self.maze[point[1]][point[0]] == '#'

  def is_empty(self, point: tuple) -> bool:
    return self.maze[point[1]][point[0]] == ' '

  def set_elem(self, point: tuple, elem: str) -> None:
    self.maze[point[1]][point[0]] = elem

  def get_elem(self, point: tuple) -> str:
    return self.maze[point[1]][point[0]]

  def height(self) -> int:
    return len(self.maze)

  def width(self) -> int:
    return len(self.maze[0]) if self.maze else 0

In [3]:
# Функция для поиска свободной позиции для аватара и ключа
def gen_position_in_maze(maze: Maze) -> tuple:
  while True:  # Ищем свободную клетку внутри лабиринта
    y = random.randint(1, maze.height() - 2)
    x = random.randint(1, maze.width() - 2)
    if maze.is_empty((x, y)):
      return x, y

In [4]:
# Функция для выбора выхода из лабиринта
def gen_exit_position(maze: Maze) -> tuple:
  exits = []
  for y in range(maze.height()):  # Проходим по левому и правому краю
    exits += [p for x in range(-1, 1) if maze.is_empty(p := (x % maze.width(), y))]
  for x in range(maze.width()):  # Проходим по верхнему и нижнему краю
    exits += [p for y in range(-1, 1) if maze.is_empty(p := (x, y % maze.height()))]
  return random.choice(exits)

In [5]:
# Функция проверки соседних клеток на наличие пути
def get_neighbors(maze: Maze, point: tuple) -> list:
  neighbors = []
  for d in [(1, 0), (-1, 0), (0, 1), (0, -1)]:  # Проверяем соседей: Справа, Слева, Снизу, Сверху
    p = (point[0] + d[0], point[1] + d[1])
    if maze.check_point(p) and not maze.is_wall(p):
      neighbors.append(p)  # Точка находится внутри лабиринта и она не является препятствием
  return neighbors

In [6]:
# Алгоритм поиска в глубину (Depth-first search)
def dfs(maze: Maze, start: tuple, key_str: str) -> list:
  stack = [(start, [start])]
  visited = set()
  while stack:
    current, path = stack.pop()
    visited.add(current)
    if maze.get_elem(current) == key_str:
      return path
    neighbors = get_neighbors(maze, current)
    for neighbor in [n for n in neighbors if n not in visited]:
      stack.append((neighbor, path + [neighbor]))
      visited.add(neighbor)

In [7]:
# Алгоритм поиска A*
def a_star_search(maze: Maze, key: tuple, exit_str: str) -> tuple:
  queue = []
  heapq.heappush(queue, (0, key, []))
  visited = set()
  while queue:
    cost, point, path = heapq.heappop(queue)
    if maze.get_elem(point) == exit_str:
      return cost, path
    if point in visited:
      continue
    visited.add(point)
    neighbors = get_neighbors(maze, point)
    for neighbor in [n for n in neighbors if n not in visited]:
      dist = abs(neighbor[0] - key[0]) + abs(neighbor[1] - key[1])
      newCost = cost + 1 + dist
      newPath = path + [neighbor]
      heapq.heappush(queue, (newCost, neighbor, newPath))

In [8]:
# Считывание лабиринта и генерации в нем точек старта, ключа и выхода.
# Функция возвращает только позицию начала пути и сам лабиринт
def get_maze(file_path: str, START: str, KEY: str, EXIT: str) -> tuple:
  maze = Maze(file_path)
  maze.set_elem(gen_exit_position(maze), EXIT)
  key = start = gen_position_in_maze(maze)
  while key == start:  # Проверяем, чтобы ключ не лежал на месте старта
    key = gen_position_in_maze(maze)
  maze.set_elem(start, START)
  maze.set_elem(key, KEY)
  return start, maze

In [9]:
# Задаем константы для вывода путей
START = 'A'
KEY = 'K'
EXIT = 'E'
TO_KEY = 'k'
TO_EXIT = 'e'
KEY_EXIT = '*'

In [10]:
start, maze = get_maze('maze-for-u.txt', START, KEY, EXIT)

In [11]:
# Поиск ключа
dfsPath = dfs(maze, start, KEY)
key_point = None
if dfsPath is not None:
  key_point = dfsPath[-1]

In [12]:
# Поиск выхода (если найден ключ)
aStarResult = None
if key_point is not None:
  aStarResult = a_star_search(maze, key_point, EXIT)

In [13]:
# Построение маршрутов
if dfsPath is not None:
  for p in dfsPath:
    if maze.is_empty(p):
      maze.set_elem(p, TO_KEY)
  if aStarResult is not None:
    for p in aStarResult[1]:
      if maze.is_empty(p):
        maze.set_elem(p, TO_EXIT)
      elif maze.get_elem(p) == 'k':
        maze.set_elem(p, KEY_EXIT)

In [14]:
# Результаты работы программы
maze.save_maze('maze-for-me-done.txt')
if key_point is not None:
  print('Путь до ключа найден!')
  if aStarResult is not None:
    print('Путь до выхода найден и его цена:', aStarResult[0])
  else:
    print('Нет выхода, стены замурованы')
else:
  print('Ключ не найден, стены замурованы')
print('Результаты путей записаны в файл maze-for-me-done.txt')

Путь до ключа найден!
Путь до выхода найден и его цена: 433246
Результаты путей записаны в файл maze-for-me-done.txt
